In [1]:
import pyspark
from pyspark import SparkFiles
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from functools import reduce

In [3]:
sc.stop()

In [4]:
# Initialize Pyspark
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)

## Import Data

In [5]:
paths = ["/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets1.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets2.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets3_1.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets3_2.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets3_3.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/users1.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/users2.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/users3.csv"]
for path in paths:
    sc.addFile(path)

tweets1 = sqlContext.read.csv(SparkFiles.get(paths[0]), header=True, inferSchema=True)
tweets2 = sqlContext.read.csv(SparkFiles.get(paths[1]), header=True, inferSchema=True)
tweets31 = sqlContext.read.csv(SparkFiles.get(paths[2]), header=True, inferSchema=True)
tweets32 = sqlContext.read.csv(SparkFiles.get(paths[3]), header=True, inferSchema=True)
tweets33 = sqlContext.read.csv(SparkFiles.get(paths[4]), header=True, inferSchema=True)

users1 = sqlContext.read.csv(SparkFiles.get(paths[5]), header=True, inferSchema=True)
users2 = sqlContext.read.csv(SparkFiles.get(paths[6]), header=True, inferSchema=True)
users3 = sqlContext.read.csv(SparkFiles.get(paths[7]), header=True, inferSchema=True)

In [6]:
tweets_df = [tweets1, tweets2, tweets31, tweets32, tweets33]
tweets = reduce(DataFrame.unionAll, tweets_df)

users_df = [users1, users2, users3]
users = reduce(DataFrame.unionAll, users_df)

In [7]:
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 

In [8]:
##### Clean dataframes ####
tweets_int_features = ["userid"]
tweets = convertColumn(tweets, tweets_int_features, IntegerType())
# drop duplicate albums
tweets = reduce(DataFrame.drop, ["follower_count", "following_count"], tweets)

users_int_features = ["userid", "follower_count", "following_count"]
users = convertColumn(users, users_int_features, IntegerType())
users.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- user_display_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_reported_location: string (nullable = true)
 |-- user_profile_description: string (nullable = true)
 |-- user_profile_url: string (nullable = true)
 |-- follower_count: integer (nullable = true)
 |-- following_count: integer (nullable = true)
 |-- account_creation_date: string (nullable = true)
 |-- account_language: string (nullable = true)



In [11]:
users.show(5)

+------+--------------------+--------------------+----------------------+------------------------+----------------+--------------+---------------+---------------------+----------------+
|userid|   user_display_name|    user_screen_name|user_reported_location|user_profile_description|user_profile_url|follower_count|following_count|account_creation_date|account_language|
+------+--------------------+--------------------+----------------------+------------------------+----------------+--------------+---------------+---------------------+----------------+
|  null|uGlAuK3VrvANhAQAi...|uGlAuK3VrvANhAQAi...|        中华人民共和国|                      🥳|            null|             1|             15|           2019-01-12|           zh-cn|
|  null|+tDWZynsin9nF3PNW...|+tDWZynsin9nF3PNW...|                  null|                    null|            null|            21|             70|           2017-09-27|           zh-cn|
|  null|NNPBUwRmLGV+945vE...|NNPBUwRmLGV+945vE...|                  null|     

In [12]:
tweets.show(5)

+--------------------+---------+-----------------+----------------+----------------------+------------------------+----------------+---------------------+----------------+--------------+--------------------+----------------+-----------------+------------------+--------------------+--------------------+----------+--------------+---------------+--------+---------+-----------+-----------+----------+-------------+--------------------+----+-------------+------------+
|             tweetid|   userid|user_display_name|user_screen_name|user_reported_location|user_profile_description|user_profile_url|account_creation_date|account_language|tweet_language|          tweet_text|      tweet_time|tweet_client_name|in_reply_to_userid| in_reply_to_tweetid|quoted_tweet_tweetid|is_retweet|retweet_userid|retweet_tweetid|latitude|longitude|quote_count|reply_count|like_count|retweet_count|            hashtags|urls|user_mentions|poll_choices|
+--------------------+---------+-----------------+----------------